In [1]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack
import pandas as pd
import numpy as np
import joblib

Using TensorFlow backend.


In [2]:
base_dir = '/kaggle/input/predictions/'

In [3]:
def get_stacked_dataset(filename_list):
    stackX = None
    for file in filename_list:
        file_path = base_dir + file
        df = pd.read_csv(file_path)
        print(df.shape)
        df = df.iloc[:, 1:5]
        yhat = np.array(df)
        print(df.shape, yhat.shape)
        if(stackX is None):
            stackX = yhat
        else:
            stackX = dstack((stackX, yhat))
    print(stackX.shape)
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

In [4]:
def fit_stacked_model(trainX, trainy):
    model = LogisticRegression()
    model.fit(trainX, trainy)
    return model

In [5]:
def train():
    trainX = get_stacked_dataset(['bert.csv', 'roberta.csv', 'xlnet.csv'])
    df = pd.read_csv(base_dir + 'roberta.csv')
    trainy = np.array(df['correct_answer'])    
    model = fit_stacked_model(trainX, trainy)
    print(model.score(trainX, trainy))
    joblib.dump(model, '/kaggle/working/log_reg.joblib')

In [6]:
train()

(11368, 6)
(11368, 4) (11368, 4)
(11368, 6)
(11368, 4) (11368, 4)
(11368, 6)
(11368, 4) (11368, 4)
(11368, 4, 3)
0.9857494722026742


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [7]:
def test():
    testX = get_stacked_dataset(['bert_test.csv', 'roberta_test.csv', 'xlnet_test.csv'])
    df = pd.read_csv(base_dir + 'roberta_test.csv')
    testy = np.array(df['correct_answer'])
    model = joblib.load('/kaggle/working/log_reg.joblib')
    print(model.score(testX, testy))

In [8]:
test()

(884, 6)
(884, 4) (884, 4)
(884, 6)
(884, 4) (884, 4)
(884, 6)
(884, 4) (884, 4)
(884, 4, 3)
0.9852941176470589
